In [1]:
#Importing Libraries
from bs4 import BeautifulSoup
import urllib3
import pandas as pd
import folium
import numpy as np
from sklearn.cluster import KMeans

### 2. Scrapping website using Beautiful Soup

In [2]:
postal_code="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"


In [3]:
import urllib.request
with urllib.request.urlopen(postal_code) as url:
    page = url.read()

In [4]:
soup = BeautifulSoup(page,'lxml')

In [5]:
name_box = soup.find('table', attrs={'class': 'wikitable sortable'})


In [6]:
row_data = []

In [7]:
 for row in name_box.find_all('tr'):
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    row_data.append(cols)
    print(cols)

[]
['M1A', 'Not assigned', 'Not assigned']
['M2A', 'Not assigned', 'Not assigned']
['M3A', 'North York', 'Parkwoods']
['M4A', 'North York', 'Victoria Village']
['M5A', 'Downtown Toronto', 'Harbourfront']
['M5A', 'Downtown Toronto', 'Regent Park']
['M6A', 'North York', 'Lawrence Heights']
['M6A', 'North York', 'Lawrence Manor']
['M7A', "Queen's Park", 'Not assigned']
['M8A', 'Not assigned', 'Not assigned']
['M9A', 'Etobicoke', 'Islington Avenue']
['M1B', 'Scarborough', 'Rouge']
['M1B', 'Scarborough', 'Malvern']
['M2B', 'Not assigned', 'Not assigned']
['M3B', 'North York', 'Don Mills North']
['M4B', 'East York', 'Woodbine Gardens']
['M4B', 'East York', 'Parkview Hill']
['M5B', 'Downtown Toronto', 'Ryerson']
['M5B', 'Downtown Toronto', 'Garden District']
['M6B', 'North York', 'Glencairn']
['M7B', 'Not assigned', 'Not assigned']
['M8B', 'Not assigned', 'Not assigned']
['M9B', 'Etobicoke', 'Cloverdale']
['M9B', 'Etobicoke', 'Islington']
['M9B', 'Etobicoke', 'Martin Grove']
['M9B', 'Etobicok

#### Data pushed in datset 

In [8]:
dataset = pd.DataFrame(row_data)
dataset= dataset.loc[1:289]
dataset=dataset.reset_index()

In [9]:
dataset=dataset[[0,1,2]]

### 3. Create dataframe
##### 3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [10]:
#Renaming the columns
dataset.columns=['PostalCode','Borough','Neighbourhood']

##### 3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [11]:
#Removing Boroughs which are not assgined
dataset= dataset[(dataset['Borough']!='Not assigned')]

In [12]:
dataset=dataset.reset_index(drop=True)

##### 3.3 More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [13]:
dataset=dataset.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
dataset.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### 3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [14]:
for row_index,row in dataset.iterrows():
    if(dataset.loc[row_index,['Neighbourhood']].values=='Not assigned'):
        dataset.loc[row_index,['Neighbourhood']]=dataset.loc[row_index,['Borough']].values

##### 3.6 Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making
Done :)

##### 3.7 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [15]:
dataset.shape

(103, 3)

### Get the latitude and the longitude coordinates of each neighborhood.
##### As Geocoder library was not responding, used csv provided

In [16]:
geocoder=pd.read_csv("/Users/inema/Desktop/delete/Geospatial_Coordinates.csv")

In [17]:
geocoder.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Adding Geo cordinates

In [18]:
dataset['Latitude'] = '0';
dataset['Longitude'] = '0';
for row_index,row in dataset.iterrows():
    for grow_index,grow in geocoder.iterrows():
        if(str(dataset.loc[row_index,['PostalCode']].values)==str(geocoder.loc[grow_index,['Postal Code']].values)):
            dataset.loc[row_index,['Latitude']]=grow.Latitude
            dataset.loc[row_index,['Longitude']]=grow.Longitude

In [19]:
dataset.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


### Explore and cluster the neighborhoods in Toronto
##### Boroughs that contain the word Toronto 

In [20]:
df = dataset.copy()
df = df[dataset.Borough.str.contains("Toronto")]
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6764,-79.293
41,M4K,East Toronto,"Riverdale, The Danforth West",43.6796,-79.3522
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.669,-79.3156
43,M4M,East Toronto,Studio District,43.6595,-79.3409
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888


##### Generate maps to visualize your neighborhoods 

In [21]:
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = df['Latitude']
Y = df['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map